## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 14.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.1.0-rc1


In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
train_images[0].shape

(28, 28, 1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9144999980926514

Best val_accuracy So Far: 0.9144999980926514
Total elapsed time: 00h 02m 36s


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        89632     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 112)               1734768   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1,826,650
Trainable params: 1,826,650
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0234 - accuracy: 0.9920 - val_loss: 0.5921 - val_accuracy: 0.9073
Epoch 5/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0248 - accuracy: 0.9911 - val_loss: 0.6308 - val_accuracy: 0.9123
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0154 - accuracy: 0.9947 - val_loss: 0.6911 - val_accuracy: 0.9127
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0191 - accuracy: 0.9932 - val_loss: 0.6827 - val_accuracy: 0.9065
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0167 - accuracy: 0.9946 - val_loss: 0.6897 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0194 - accuracy: 0.9936 - val_loss: 0.6115 - val_accuracy: 0.9173
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0135 - accuracy: 0.9955 - val_loss: 0.7371 - val_accurac